In [23]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=8, app_name='west0_workshop', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_workshop...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:36


In [22]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [25]:
pc_patch = {
    "18.2": ["2022-07-13", "2022-08-09"],
    "19.1": ["2022-08-10", "2022-09-06"],
    "19.2": ["2022-09-07", "2022-10-11"],
    "20.1": ["2022-10-12", "2022-11-08"]
}

console_patch = {
    "18.2": ["2022-07-21", "2022-08-17"],
    "19.1": ["2022-08-18", "2022-09-14"],
    "19.2": ["2022-09-15", "2022-10-19"],
    "20.1": ["2022-10-20", "2022-11-16"]
}

# pc_patch = {
#     "18.2": ["2022-07-13", "2022-08-09"],
#     "19.1": ["2022-08-10", "2022-09-06"],
#     "19.2": ["2022-09-07", "2022-10-11"],
#     "20.1": ["2022-10-12", "2022-11-08"],
#     "20.2": ["2022-11-09", "2022-12-05"]
# }

# console_patch = {
#     "18.2": ["2022-07-21", "2022-08-17"],
#     "19.1": ["2022-08-18", "2022-09-14"],
#     "19.2": ["2022-09-15", "2022-10-19"],
#     "20.1": ["2022-10-20", "2022-11-16"],
#     "20.2": ["2022-11-17", "2022-12-14"]  
# }

In [28]:
pc_df = {
    "patch": [],
    "au": [],
    "paid_pu": [],
    "total_pu": [],
    "paid_use": [],
    "total_use": []
}

for patch, date in pc_patch.items():
    start_date = date[0]
    end_date = date[1]
    au = load_data_mart("pc", end_date, end_date, "user_master").where(col("lastlogindate") >= start_date).select(countDistinct("accountid")).collect()[0][0]
    gcoin = load_data_mart("pc", start_date, end_date, "gcoin_use").where(col("event_name") == "202207_workshop") \
        .withColumn("paid_account_id", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None)))
    paid_pu = gcoin.select(countDistinct("paid_account_id")).collect()[0][0]
    total_pu = gcoin.select(countDistinct("account_id")).collect()[0][0]
    paid_use = gcoin.select(sum("paid_use")).collect()[0][0]
    total_use = gcoin.select(sum(col("paid_use") + col("free_use"))).collect()[0][0]
    
    pc_df["patch"].append(patch)
    pc_df["au"].append(au)
    pc_df["paid_pu"].append(paid_pu)
    pc_df["total_pu"].append(total_pu)
    pc_df["paid_use"].append(paid_use)
    pc_df["total_use"].append(total_use)

In [29]:
pc_pd = pd.DataFrame.from_dict(pc_df)

In [30]:
pc_pd

,patch,au,paid_pu,total_pu,paid_use,total_use
0,18.2,9660517,289044,677314,1874838675,2341605700
1,19.1,9349556,217104,496025,822832986,1087156860
2,19.2,11048805,118306,282129,267465330,388553520
3,20.1,8557149,107493,280646,211822685,322723130


In [31]:
console_df = {
    "patch": [],
    "au": [],
    "paid_pu": [],
    "total_pu": [],
    "paid_use": [],
    "total_use": []
}

for patch, date in console_patch.items():
    start_date = date[0]
    end_date = date[1]
    au = load_data_mart("console", end_date, end_date, "user_master").where(col("lastlogindate") >= start_date).select(countDistinct("accountid")).collect()[0][0]
    gcoin = load_data_mart("console", start_date, end_date, "gcoin_use").where(col("event_name") == "202207_workshop") \
        .withColumn("paid_account_id", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None)))
    paid_pu = gcoin.select(countDistinct("paid_account_id")).collect()[0][0]
    total_pu = gcoin.select(countDistinct("account_id")).collect()[0][0]
    paid_use = gcoin.select(sum("paid_use")).collect()[0][0]
    total_use = gcoin.select(sum(col("paid_use") + col("free_use"))).collect()[0][0]
    
    console_df["patch"].append(patch)
    console_df["au"].append(au)
    console_df["paid_pu"].append(paid_pu)
    console_df["total_pu"].append(total_pu)
    console_df["paid_use"].append(paid_use)
    console_df["total_use"].append(total_use)

In [32]:
console_pd = pd.DataFrame.from_dict(console_df)

In [33]:
console_pd

,patch,au,paid_pu,total_pu,paid_use,total_use
0,18.2,1714847,28866,56685,87631180,119930980
1,19.1,1510832,19267,39333,52242410,74210790
2,19.2,1662100,10554,19954,21279110,30154940
3,20.1,1386610,9201,19707,17255650,25621140


In [35]:
pc_cum_gcoin = load_data_mart("pc", "2022-07-13", "2022-11-08", "gcoin_use").where(col("event_name") == "202207_workshop")

In [36]:
# user 지역
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())

meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

user = load_data_mart("pc", "2022-11-08", "2022-11-08", "user_master")
user = user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
user = user.join(meta_region, user.country_new == meta_region.country_code_iso2, "left")

In [37]:
user = user.withColumnRenamed("accountid", "account_id").select("account_id", "pubg_region")
pc_cum_gcoin = pc_cum_gcoin.join(user, "account_id", "left")

In [38]:
pc_cum_gcoin.groupBy("pubg_region").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).toPandas()

,pubg_region,paid_use,total_use
0,CN,2471531185,3138902550
1,NA,61630610,87251590
2,SA,12961120,32332680
3,KR,405666480,512045230
4,SEA,32961680,52171180
5,Undefined,0,2930
6,JP,4895320,6821600
7,TW/HK,15418380,19481270
8,CIS,54132571,100815370
9,EMEA,103020300,170599880


In [39]:
console_cum_gcoin = load_data_mart("console", "2022-07-21", "2022-11-16", "gcoin_use").where(col("event_name") == "202207_workshop")
console_user = load_data_mart("console", "2022-11-16", "2022-11-16", "user_master")
console_user = console_user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
console_user = console_user.join(meta_region, console_user.country_new == meta_region.country_code_iso2, "left")

In [40]:
console_user = console_user.withColumnRenamed("accountid", "account_id").select("account_id", "pubg_region")
console_cum_gcoin = console_cum_gcoin.join(console_user, "account_id", "left")

In [41]:
console_cum_gcoin.groupBy("pubg_region").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).toPandas()

,pubg_region,paid_use,total_use
0,CN,1328550,1700480
1,NA,119073840,163419180
2,SA,8271230,13005100
3,KR,1666650,2136280
4,SEA,389720,523780
5,JP,4535240,5709240
6,TW/HK,1314940,1609780
7,CIS,1630370,5084920
8,EMEA,32720990,46877750
9,OC,7476820,9851340


In [18]:
load_data_mart("console", "2022-07-27", "2022-07-27", "user_master").where(col("lastlogindate") >= "2022-07-21").select(countDistinct("accountid").alias("au")).show(truncate=False)

+------+
|au    |
+------+
|739339|
+------+



In [17]:
workshop_console_gcoin = load_data_mart("console", "2022-07-21", "2022-07-27", "gcoin_use").where(col("event_name") == "202207_workshop")

In [19]:
workshop_console_gcoin = workshop_console_gcoin.withColumn("paid_account_id", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None)))

In [20]:
workshop_console_gcoin.select(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show(truncate=False)

+--------+---------+
|paid_use|total_use|
+--------+---------+
|33123100|46028640 |
+--------+---------+



In [21]:
workshop_console_gcoin.select(countDistinct("paid_account_id").alias("paid_pu"), countDistinct("account_id").alias("total_pu")).show(truncate=False)

+-------+--------+
|paid_pu|total_pu|
+-------+--------+
|14327  |26681   |
+-------+--------+

